# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='./Reacher_multiagent.app', no_graphics=True)

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    #print("actions shape", actions.shape)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    print(rewards)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print(scores)
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from collections import deque, namedtuple
from typing import List, Tuple
from unityagents import UnityEnvironment
import os
import random

# Models

In [2]:
class OUNoise:
    def reset(self):
        self.state = self.mu
        
    def __init__(self, size:int, mu:float=0, theta:float=0.15, sigma:float=0.1):
        self.mu = np.ones(size) * mu
        self.theta = theta
        self.sigma = sigma
        self.reset()
        
    def sample(self):
        x = self.state
        dx = (x - self.mu) * self.theta + self.sigma * np.array([np.random.rand() for i in range(len(x))])
        self.state += dx
        return self.state

In [3]:
class ValueNetwork(nn.Module):
    def __init__(self, state_size:int, action_size:int):
        super().__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        self.layers = nn.Sequential(
            nn.Linear(self.state_size + self.action_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
    def forward(self, s:torch.Tensor, a:torch.Tensor)->torch.Tensor:
        x = torch.concatenate((s, a), axis=1)
        return self.layers(x)

In [4]:
class OptimalActionNetwork(nn.Module):
    def __init__(self, state_size: int, action_size:int):
        super().__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.layers = nn.Sequential(
            nn.Linear(self.state_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, self.action_size),
            nn.Tanh()
        )
    def forward(self, x:torch.tensor)->torch.tensor:
        return self.layers(x)

# Utils

In [5]:
def soft_update(target:nn.Module, source:nn.Module, fraction:float=0.005):
    for t,s in zip(target.parameters(), source.parameters()):
        t = (1-fraction)*t + fraction * s
        
def concatenate_history(lst:List[np.ndarray]):
    return np.concatenate(lst, axis=1)

def initialize_seeds(seed:int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

# Data storage

In [6]:
Experience = namedtuple('Experience',['state', 'action', 'reward', 'next_state', 'done'])
class ReplayBuffer:
    def __init__(self, buffer_size:int, batch_size:int):
        assert buffer_size > 0 and buffer_size >= batch_size
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.memory = deque(maxlen=self.buffer_size)
        
    def add(self, state, action, reward, next_state, done):
        self.memory.append(Experience(state, action, reward, next_state, done))
        
    def __len__(self)->int:
        return len(self.memory)
    
    def convert_to_tensors(self, experiences:Tuple[List[np.ndarray], List[np.ndarray], List[np.ndarray],
                                                   List[np.ndarray], List[np.ndarray]],
                                                   device:str='cpu')->Tuple[torch.tensor, torch.tensor, torch.tensor,
                                                                           torch.tensor, torch.tensor]:
        return (torch.from_numpy(experiences[0]).float().to(device),
                torch.from_numpy(experiences[1]).float().to(device),
                torch.from_numpy(experiences[2]).float().to(device),
                torch.from_numpy(experiences[3]).float().to(device),
                torch.from_numpy(experiences[4]).float().to(device))
        
    def sample_experiences(self, indices:List[int])->Tuple[List[np.ndarray], List[np.ndarray], List[np.ndarray],
                                                   List[np.ndarray], List[np.ndarray]]:
        experiences = [self.memory[i] for i in indices]
        states = np.vstack([e.state for e in experiences])
        actions = np.vstack([e.action for e in experiences])
        rewards = np.vstack([e.reward for e in experiences])
        next_states = np.vstack([e.next_state for e in experiences])
        dones = np.vstack([e.done for e in experiences])
        return (states, actions, rewards, next_states, dones)
        

        
    def sample(self, num_steps:int, device:str='cpu')->List[Tuple[torch.tensor, torch.tensor, torch.tensor,
                                                   torch.tensor, torch.tensor]]:
        indices = [i for i in range(len(self) + 1 - num_steps)]
        indices = indices[:self.batch_size]
        random.shuffle(indices)
        samples = []
        for i in range(num_steps):
            samples.append(self.convert_to_tensors(self.sample_experiences(indices)))
            indices = [i + 1 for i in indices]
        
        return samples
            
                          

# Config

In [7]:
DEFAULT_BUFFER_SIZE=1024
DEFAULT_BATCH_SIZE=64
DEFAULT_GAMMA=0.99
DEFAULT_TAU=1e-3
DEFAULT_HIST_SIZE=4
DEFAULT_NSTEPS=3
DEFAULT_ACTOR_LR=1e-4
DEFAULT_CRITIC_LR=1e-3
DEFAULT_ACTOR_UPDATE_EVERY=10

SOLVE_SCORE_THRESHOLD=30
SOLVE_SCORE_WINDOW=100
NUM_EPISODES=10000
REPORT_EVERY=100

# Agent

In [8]:
class D4PGAgent:
    def __init__(self, env:UnityEnvironment, device:str='cpu', buffer_size:int=DEFAULT_BUFFER_SIZE,
                 batch_size:int=DEFAULT_BATCH_SIZE, nsteps:int=DEFAULT_NSTEPS, gamma:float=DEFAULT_GAMMA,
                 tau:float=DEFAULT_TAU, update_actor_every:int=DEFAULT_ACTOR_UPDATE_EVERY,
                 hist_size:int=DEFAULT_HIST_SIZE, actor_lr:float=DEFAULT_ACTOR_LR, critic_lr:float=DEFAULT_CRITIC_LR):
        self.env = env
        self.min_action = -1
        self.max_action = 1
        brain_name = env.brain_names[0]
        brain = env.brains[brain_name]
        env_info = env.reset(train_mode=True)[brain_name]

        # number of agents
        self.num_agents = len(env_info.agents)
        self.action_size = brain.vector_action_space_size
        states = env_info.vector_observations
        self.state_size = states.shape[1]
        self.device = device
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.nsteps = nsteps
        self.gamma = gamma
        self.tau = tau
        self.critic_lr = critic_lr
        self.actor_lr = actor_lr
        self.hist_size = hist_size
        self.update_actor_every = update_actor_every
        self.iteration = 0
        self.buffer = ReplayBuffer(self.buffer_size, self.batch_size)
        self.actor = OptimalActionNetwork(self.state_size*self.hist_size, self.action_size)
        self.actor_target = OptimalActionNetwork(self.state_size*self.hist_size, self.action_size)
        self.critic = ValueNetwork(self.state_size*self.hist_size, self.action_size)
        self.critic_target = ValueNetwork(self.state_size*self.hist_size, self.action_size)
        soft_update(self.actor_target, self.actor, 1)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=self.actor_lr)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=self.critic_lr)
        self.noise = OUNoise(self.action_size)
        
    def choose_action(self, state:np.ndarray, add_noise:bool=True)->np.ndarray:
        assert(state.shape[1] == self.state_size * self.hist_size)
        state_t = torch.from_numpy(state).float().to(self.device)
        self.actor.eval()
        with torch.no_grad():
            action = self.actor(state_t).cpu().numpy()
        self.actor.train()
        action = action * self.max_action
        if add_noise:
            action += self.noise.sample()
        
        return np.clip(action, self.min_action, self.max_action)
    
    def learn(self):
        samples = self.buffer.sample(self.nsteps, self.device)
        gamma = self.gamma
        rewards = samples[0][2] * samples[0][-1]
        for i in range(1, self.nsteps):
            gamma = gamma * self.gamma
            rewards + samples[i][2] * gamma * samples[i][-1]
        next_actions = self.actor_target(samples[-1][-2]) * self.max_action
        next_actions = next_actions.clip(self.min_action, self.max_action)
        q_targets = rewards + gamma * self.gamma * self.critic_target(samples[-1][-2], next_actions)
        q_expected = self.critic(samples[0][0], samples[0][1])
        assert len(q_targets) == len(q_expected)
        
        critic_loss = F.mse_loss(q_expected, q_targets)
        
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        
        if self.iteration % self.update_actor_every == 0:
            actions_pred = self.actor(samples[0][0]) * self.max_action
            actions_pred = actions_pred.clip(self.min_action, self.max_action)
            actor_loss = -self.critic(samples[0][0], actions_pred)
            for i in range(1, self.nsteps):
                actions_pred = self.actor(samples[i][0]) * self.max_action
                actions_pred.clip(self.min_action, self.max_action)
                actor_loss = actor_loss -self.critic(samples[i][0], actions_pred)
            actor_loss = actor_loss.mean()
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()
        
        
    
    def step(self, states:np.ndarray, actions:np.ndarray, rewards:float, next_states:np.ndarray, dones:bool):
        assert states.shape[0] == self.num_agents
        assert states.shape[1] == self.state_size * self.hist_size
        assert next_states.shape[1] == self.state_size * self.hist_size
        assert len(states) == len(actions)
        assert len(states) == len(rewards)
        assert len(states) == len(next_states)
        assert len(states) == len(dones)
        self.iteration += 1
        for i in range(len(states)):
            self.buffer.add(states[i], actions[i], rewards[i], next_states[i], dones[i])
        if len(self.buffer) >= self.batch_size:
            self.learn()
            
    
    def save(self, dir:str):
        torch.save(self.actor.model_dict, os.path.join(dir, "actor.pt"))
        torch.save(self.critic.model_dict, os.path.join(dir, "critic.pt"))
        
    def load(self, dir:str):
        self.actor.load_state_dict(torch.load(os.path.join(dir, "actor.pt")))
        self.actor_target.load_state_dict(torch.load(os.path.join(dir, "actor.pt")))
        self.critic.load_state_dict(torch.load(os.path.join(dir, "critic.pt")))
        self.critic_target.load_state_dict(torch.load(os.path.join(dir, "critic.pt")))

# run script

In [ ]:
initialize_seeds(0x42)
scores = []
score_window = deque(maxlen=SOLVE_SCORE_WINDOW)
env = UnityEnvironment(file_name='./Reacher_multiagent.app', no_graphics=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
hist_size = DEFAULT_HIST_SIZE
agent = D4PGAgent(env, device, update_actor_every=20)
total_steps = 0
for i in range(1, NUM_EPISODES + 1):

    
    #if i % REPORT_EVERY-1 == 0:
    print(f"staring {i}th episode avg score {np.mean(score_window)} {np.mean(scores)} training steps {total_steps}", end='\r')
    hist = deque(maxlen=hist_size)
    env_info = env.reset()[env.brain_names[0]]
    #print("env info is ", env_info)
    hist.append(env_info.vector_observations)
    while (len(hist) < hist_size):
        hist.append(env_info.vector_observations)
    states = concatenate_history(list(hist))
    score = np.zeros(agent.num_agents)
    total_steps = 0
    while True:
        total_steps += 1
        actions = agent.choose_action(states)
        #print("actions shape", actions.shape, "states shape", states.shape)
        env_info = env.step(actions)[env.brain_names[0]]
        rewards = env_info.rewards
        mean_rewards = np.mean(rewards)
        score += np.array(rewards)
        #scores.append(mean_rewards)
        #score_window.append(mean_rewards)
        hist.append(env_info.vector_observations)
        next_states = concatenate_history(list(hist))
        dones = env_info.local_done
        agent.step(states,actions, rewards, next_states, dones)
        states = next_states
        if np.any(dones):
            #print(score)
            scores.append(score)
            score_window.append(score)
            break
    if len(score_window) < SOLVE_SCORE_WINDOW:
        continue
    
    if np.mean(score_window) > SOLVE_SCORE_THRESHOLD:
        print(f"Solved environment in {i} episodes with score {np.mean(score_window)}")
        agent.save("./")
        break
    

fit = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Mono path[0] = '/Users/rkhona/learn/deeprl/control_project/Reacher_multiagent.app/Contents/Resources/Data/Managed'
Mono config path = '/Users/rkhona/learn/deeprl/control_project/Reacher_multiagent.app/Contents/MonoBleedingEdge/etc'


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
/Users/rkhona/opt/anaconda3/envs/drlnd/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/rkhona/opt/anaconda3/envs/drlnd/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
